In [1]:
# Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

save_initial = False

In [2]:
# Aggregate all seasons into one directory
if save_initial:
    input_paths    = [f"./data/20{i}-{i+1}/players/" for i in range(16, 23)]
    init_path      = "./data/2023-24/players/"
    save_path      = "./data/2016-24/"
    for i in os.listdir(init_path):
        if i[0] == '.':
            continue
        df         = pd.read_csv(os.path.join(init_path, i, "gw.csv"))
        for j in input_paths:
            for k in os.listdir(j):
                if k.split("_")[:-1] != i.split("_")[:-1]:
                    continue
                d2 = pd.read_csv(os.path.join(j, k, "gw.csv"))
                df = pd.concat([df, d2], ignore_index=True)
        df.to_csv(os.path.join(save_path, f'{"_".join(i.split("_")[:-1])}.csv'), index=False)

In [3]:
# Make cummulative frequency curve of games listed per player
# Figure out what columns are not present in all dfs

if save_initial:
    dfs            = []
    cols           = set()
    not_all        = set()
    for i in os.listdir(save_path):
        dfs       += [pd.read_csv(os.path.join(save_path, i))]
        cols       = cols.union(set(dfs[-1].columns))
        not_all    = not_all.union(set(cols).difference(set(dfs[-1].columns)))

    numGames       = [len(i) for i in dfs]
    total          = max(numGames)
    freq           = [0] * (total + 1)
    for i in numGames:
        freq[i]   += 1
    temp           = 0
    for i in range(len(frequency) - 1, -1, -1):
        freq[i]   += temp
        temp       = freq[i]
    plt.plot(range(len(freq)), freq)

    print(not_all)

In [4]:
# Create training dataset: 
    #    train.csv containing (nx5m values), where each row is a flattened stat summary from past 5 games for one player
    #    labels.csv containing (nx1) values, total points in next game for each player
    
not_all      = {'target_missed', 'fouls', 'recoveries', 'key_passes', 
                'clearances_blocks_interceptions', 'big_chances_created', 
                'errors_leading_to_goal_attempt', 'completed_passes', 'tackled', 
                'kickoff_time_formatted', 'attempted_passes', 'tackles', 'offside', 
                'big_chances_missed', 'loaned_in', 'errors_leading_to_goal', 'penalties_conceded', 
                'loaned_out', 'winning_goals', 'open_play_crosses', 'dribbles', 'id', 'ea_index'}
columnNames  = ['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
                'expected_assists', 'expected_goal_involvements', 'expected_goals',
                'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
                'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
                'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
                'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
                'threat', 'total_points', 'transfers_balance', 'transfers_in',
                'transfers_out', 'value', 'was_home', 'yellow_cards',
                'attempted_passes', 'big_chances_created', 'big_chances_missed',
                'clearances_blocks_interceptions', 'completed_passes', 'dribbles',
                'ea_index', 'errors_leading_to_goal', 'errors_leading_to_goal_attempt',
                'fouls', 'id', 'key_passes', 'kickoff_time_formatted', 'loaned_in',
                'loaned_out', 'offside', 'open_play_crosses', 'penalties_conceded',
                'recoveries', 'tackled', 'tackles', 'target_missed', 'winning_goals']
drop_columns = {'kickoff_time', 'transfers_balance', 'fixture', 'creativity', 'element',
                'expected_assists', 'expected_goal_involvements', 'expected_goals',
                'fixture', 'ict_index', 'influence', 'starts', 'team_a_score', 
                'threat', 'team_h_score', 'expected_goals_conceded', 'opponent_team', 'was_home'}.union(not_all)

columnNames  = [i for i in columnNames if i not in drop_columns]
print(columnNames)

['assists', 'bonus', 'bps', 'clean_sheets', 'goals_conceded', 'goals_scored', 'minutes', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves', 'selected', 'total_points', 'transfers_in', 'transfers_out', 'value', 'yellow_cards']


In [5]:
# NaN analysis
input_path       = "./data/2016-24/"
dfs              = [pd.read_csv(os.path.join(input_path, i)).loc[:, columnNames] 
                    for i in os.listdir(input_path)
                    if i[0] != "."]
megaDf           = pd.concat(dfs, axis=0, ignore_index=True)
counts           = megaDf.isna().sum()
print(counts)
# Insert NaN handling if NaNs

# Normalization
save_path        = "./data/2016-24_processed/"
means            = {col:count for col, count in megaDf.mean().items()}
sds              = {col:count for col, count in megaDf.std().items()}
print(means)
print(sds)

# Don't normalize total_points
means['total_points'] = 0
sds['total_points'] = 1

for name, df in zip(os.listdir(input_path), dfs):
    for col, mean in means.items():
        df[col] -= mean
        df[col] /= sds[col]
    df.to_csv(os.path.join(save_path, name), index = False)

assists             0
bonus               0
bps                 0
clean_sheets        0
goals_conceded      0
goals_scored        0
minutes             0
own_goals           0
penalties_missed    0
penalties_saved     0
red_cards           0
round               0
saves               0
selected            0
total_points        0
transfers_in        0
transfers_out       0
value               0
yellow_cards        0
dtype: int64
{'assists': 0.047924781349427154, 'bonus': 0.12616815762815847, 'bps': 6.874973248298592, 'clean_sheets': 0.11929118691948808, 'goals_conceded': 0.5609154075532538, 'goals_scored': 0.05170568848178744, 'minutes': 36.758507041047814, 'own_goals': 0.001940390075475467, 'penalties_missed': 0.000955927463653355, 'penalties_saved': 0.0006848435560501648, 'red_cards': 0.0020259955199817374, 'round': 18.842100757608183, 'saves': 0.120004565623707, 'selected': 258238.08733182098, 'total_points': 1.5686056299847337, 'transfers_in': 18494.909957339954, 'transfers_out': 166

In [6]:
trainColumns   = []
for j in ["_1", "_2", "_3", "_4", "_5"]: # 5 games
    trainColumns += [i+j for i in columnNames]

train          = []
labels         = []

input_path     = "./data/2016-24_processed/"
for i in tqdm(os.listdir(input_path)):
    if i[0] == '.':
        continue
    curr_path  = os.path.join(input_path, i)
    df         = pd.read_csv(curr_path)
    for label_row in range(5, len(df)):
        vector = df.iloc[label_row - 5:label_row].values.flatten()
        label  = df['total_points'].iloc[label_row]
        train += [pd.Series(vector, index = trainColumns)]
        labels+= [pd.Series([label], index = ["Points"])]
        
train          = pd.concat(train, ignore_index = True)
labels         = pd.concat(labels, ignore_index = True)

100%|██████████| 812/812 [00:14<00:00, 56.88it/s]


In [7]:
save_path      = "./train_2016-24/"
train.to_csv(os.path.join(save_path, 'inputs.csv'), index=False)
labels.to_csv(os.path.join(save_path, 'labels.csv'), index=False)